In [ ]:
from LSTM import *

In [ ]:
covid_state, state_list = load_states()
print(covid_state["OH"].head())

ts = TimeSeries()
ts.setDataFrame(covid_state["AZ"][1:])
ts.setColumns(["beds","cases","vaccines","cases_7day"])
ts.splitData()
ts.normalizeData()
ts.displayColumns(["beds"])
ts.displayColumns(["cases"])

In [ ]:
ts.setWindowLength(label_columns=["beds"])
ts.setDataset(batch_size=32)
ts.setWindowLength(input_width = 28, label_width = 10, shift = 10,label_columns = ["beds"])
ts.generateWindow()
#ts.createWindow()
#ts.train.element_spec

In [ ]:
for example_inputs, example_labels in ts.test.take(1):
    print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
    print(f'Labels shape (batch, time, features): {example_labels.shape}')
    
example_labels = tf.reshape(example_labels,[example_labels.shape[0],-1])
example_labels

In [ ]:
column_indices = ts.getColumnIndices()
print(column_indices)

In [ ]:
class Baseline(tf.keras.Model):
    def __init__(self, label_index=None):
        super().__init__()
        self.label_index = label_index

    def call(self, inputs):
        if self.label_index is None:
            return inputs
        result = inputs[:, :, self.label_index]
        return result[:, :, tf.newaxis]
    
baseline = Baseline(label_index=column_indices['beds'])
baseline.compile(loss=tf.losses.MeanSquaredError(),
                 metrics=[tf.metrics.MeanAbsoluteError()])
val_performance = {}
performance = {}
ts.val
val_performance['Baseline'] = baseline.evaluate(ts.val)
performance['Baseline'] = baseline.evaluate(ts.test, verbose=0)

In [ ]:
MAX_EPOCHS = 50
def compile_and_fit(model, window, patience=2):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')
    model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])
    history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
    return history

import IPython
import IPython.display    


In [ ]:
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=False),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=10),
    #tf.keras.layers.Dense(units=2),
    #tf.keras.layers.Reshape((None,2,1))
])

history = compile_and_fit(lstm_model, ts)

#IPython.display.clear_output()
val_performance = {}
performance = {}
val_performance['LSTM'] = lstm_model.evaluate(ts.val)
performance['LSTM'] = lstm_model.evaluate(ts.test, verbose=0)

ts.plot(lstm_model)


In [ ]:
inputs, labels = next(iter(ts.train))
labels
lstm_model.predict(inputs)

In [ ]:
plot_col_index = ts.column_indices["beds"]
inputs,labels = ts.example
plt.plot(ts.input_indices, inputs[0, :, 1],label='Inputs', marker='.', zorder=-10)


In [ ]:
plt.scatter(ts.label_indices, labels[0, -(len(ts.label_indices)):, 0],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)

In [ ]:
ts.plot(lstm_model,plot_col='beds', max_subplots=10)

In [ ]:
ts.DataFrame
ts.DataFrame.drop(columns="day_number")
#ts.DataFrame